In [14]:
import numpy as np

from skimage import io, transform
from SIMulator_functions import *
import glob
import os
import argparse

In [15]:
# ------------ Parameters-------------
def GetParams():
    opt = argparse.Namespace()

    # phase shifts for each stripe
    opt.Nshifts = 3
    # number of orientations of stripes
    opt.Nangles = 3
    # modulation factor
    opt.ModFac = 0.3+0.4*np.random.rand()
    # orientation offset
    opt.alpha = pi/3*(np.random.rand()-0.5)
    # orientation error
    opt.angleError = 10*pi/180*(np.random.rand()-0.5)
    # shuffle the order of orientations
    opt.shuffleOrientations = False
    # random phase shift errors
    opt.phaseError = 1*pi*(0.5-np.random.rand(opt.Nangles, opt.Nshifts))
    # in percentage
    opt.NoiseLevel = 18 + 8*(np.random.rand()-0.5)
    # Poisson noise extent
    opt.Poisson = np.random.randint(10000,500000)
    # include OTF and GT in stack
    opt.OTF_and_GT = True
    # NA
    opt.NA = 1.2
    # Emission wavelength
    opt.emission = np.random.randint(520,680)
    # Pattern frequency  
    kMax = (2*opt.NA)/(opt.emission)
    opt.k2 = 0.8*(kMax)
    # Pixel Size
    opt.Psize = np.random.randint(80,110)
    
    return opt


In [16]:
# ------------ Options --------------
nrep = 3

sLoc = "E:/Users/ew535/Training datasets/Trained sets/16-06-2020/"
files = glob.glob('E:/Users/ew535/Training datasets/DIV2K raw/DIV2K_train_HR/*.png')
files[0]

'E:/Users/ew535/Training datasets/DIV2K raw/DIV2K_train_HR\\0001.png'

In [17]:
Io = io.imread(files[0]) / 255
Io = Io - 0.3*np.amax(Io)
Io[Io<0] = 0
print(np.amin(Io[:,:,0].shape))

1404


In [18]:
from IPython import display
n_rep = 5
sNum = 1
for n_it in range(n_rep):
    # I think there was an nRep loop here to go through the data set multiple times
    # ------------ Main loop --------------
    for file in files:
        Io = io.imread(file) 
        Io = Io / np.amax(Io)
        Io = Io - 0.3 # Cropping LUT like this increases the number of sharp details in the image 
        Io[Io<0] = 0
        Io = Io/np.amax(Io)
        
        minDim = np.amin(Io[:,:,0].shape)
        # This is a very ugly way of re-sampling regions of the image
        if (n_it % 2) ==0:
            Io = Io[0:minDim,0:minDim,:]
           
        else:
            Io = np.rot90(Io,2)
            Io = Io[0:minDim,0:minDim,:]
            
        Io = transform.resize(Io, (512, 512), anti_aliasing=True)

        if np.random.rand(1) > 0.65: # 35 percent  of the time use the same image for background light

            # Use same image
            Oi = Io[:,:,np.random.randint(1,3)]  # if not grayscale
            Io = Io[:,:,0]

        else:
            # Use another image
            if  Io.shape[2] > 1:
                Io = Io.mean(2)

            fnew = files[np.random.randint(0, len(files))]
            Oi = io.imread(fnew) 
            Oi = Oi / np.amax(Oi)
            Oi = Oi - 0.3
            Oi[Oi<0] = 0

            minDim = np.amin(Oi[:,:,0].shape)
            Oi = Oi[0:minDim-1,0:minDim-1,:]
            Oi = transform.resize(Oi, (512, 512), anti_aliasing=True)

            if  Oi.shape[2] > 1: # if not grayscale
                Oi = Oi.mean(2)  
                Oi = Oi/np.amax(Oi)

        Io = np.rot90(Io,n_it)
        Oi = np.rot90(Oi,n_it)
        opt = GetParams()
        stack = Generate_SIM_Image(opt, Io, Oi)

        # normalise
        for i in range(len(stack)):
            stack[i] = (stack[i] - np.min(stack[i])) / \
                (np.max(stack[i]) - np.min(stack[i]))

        stack = (stack * 255).astype('uint8')

        svPath = sLoc+str(sNum)+".tif"
        io.imsave(svPath,stack)
        
        if np.remainder(sNum,20) == 0:        
            print('Done image',sNum, '/', n_rep*len(files))
            
        sNum += 1




Done image 20 / 4500
Done image 40 / 4500
Done image 60 / 4500
Done image 80 / 4500
Done image 100 / 4500
Done image 120 / 4500
Done image 140 / 4500
Done image 160 / 4500
Done image 180 / 4500
Done image 200 / 4500
Done image 220 / 4500
Done image 240 / 4500
Done image 260 / 4500
Done image 280 / 4500
Done image 300 / 4500
Done image 320 / 4500
Done image 340 / 4500
Done image 360 / 4500
Done image 380 / 4500
Done image 400 / 4500
Done image 420 / 4500
Done image 440 / 4500
Done image 460 / 4500
Done image 480 / 4500
Done image 500 / 4500
Done image 520 / 4500
Done image 540 / 4500
Done image 560 / 4500
Done image 580 / 4500
Done image 600 / 4500
Done image 620 / 4500
Done image 640 / 4500
Done image 660 / 4500
Done image 680 / 4500
Done image 700 / 4500
Done image 720 / 4500
Done image 740 / 4500
Done image 760 / 4500
Done image 780 / 4500
Done image 800 / 4500
Done image 820 / 4500
Done image 840 / 4500
Done image 860 / 4500
Done image 880 / 4500
Done image 900 / 4500
Done image 920

100